In [ ]:
print("Settings.py should be e.g. cnn_flows_prtrain")
#DESIGN PARAMETERS FOR NEURAL NETWORK

VALIDATION_SPLIT = 0.2 #indicated ratio of training to validation data: 0.2 -> 20% VALIDATION data
RANDOMIZED_SEED = 20
SHUFFLE_DATASET = False

TRAINING_PREPARATION = False

BATCH_SIZE = 32
TIME_STEPS = 40

NR_EPOCHS = 50

LR = 0.001
# MOMENTUM = 0.9
print("Hyperparameters defined")
print("Bug with learning rate when sequence length is not the same for each element of the dataset")

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np
import time
import os
import os.path as path
import copy

from gait_analysis import AnnotationsCasia as Annotations
from gait_analysis import CasiaDataset
from gait_analysis.Config import Config
from gait_analysis import Composer
from gait_analysis import WeightWatcher
from gait_analysis import AccuracyTracker
from gait_analysis import AccuracyTrackerTrainTest

import matplotlib.pyplot as plt

from gait_analysis.Models import PretrainConvFlow

print("done")

In [ ]:
#change configuration in settings.py
c = Config()
c.config['indexing']['grouping'] = 'person_sequence_angle'
composer = Composer()
transformer = composer.compose()
dataset = CasiaDataset(transform=transformer)
dataset_size = len(dataset)
indices = list(range(dataset_size))
print("Dataset size:", len(dataset))
print("Indices size:", len(indices))
split = int(np.floor(VALIDATION_SPLIT * dataset_size))
print("Split:", split)
if SHUFFLE_DATASET:
    np.random.seed(RANDOMIZED_SEED)
    np.random.shuffle(indices)
train_indices, test_indices = indices[split:], indices[:split]
train_sampler = torch.utils.data.SequentialSampler(train_indices)
test_sampler = torch.utils.data.SequentialSampler(test_indices)

train_loader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, sampler=train_sampler)
test_loader = torch.utils.data.DataLoader(dataset, batch_size=1, sampler=test_sampler)

In [ ]:
net = PretrainConvFlow.PretrainConvFlow()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
net.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=LR)

learning_rate_array = []
loss_array = []
# monitor = WeightWatcher(net,['conv5','fc1'])
tt_acc_tracker = AccuracyTrackerTrainTest([0,1,2])
# test_acc_tracker = AccuracyTracker([0,1,2])

In [ ]:
# if TRAINING_PREPARATION:
#     print('Start training...')
#     print("Expected loss with {} different classes and {} data elements: {}".format(3, len(dataset)-split, (len(dataset)-split)*np.log(3)))
#     running_loss = 0.0
#     print("Batch size:", BATCH_SIZE)
#     print("Evaluating first element...")
#     start_time = time.time()
#     i, batch = next(iter(enumerate(train_loader)))
#     inputs, labels = batch
#     #TODO make list disappear

#     data_in_list = [s.to(device) for s in inputs['flows']]
#     labels = labels.to(device)

#     print("--------------------------\nOriginal input:")
#     print("Data in: List length", len(data_in_list), "List element:", data_in_list[0].size())
#     print("Labels:", labels.size())
#     print("--------------------------\nSingle element:")
#     data_element = data_in_list[0]
#     label_element = labels[:,0].long()
#     print("Data element:", data_element.size())
#     print("Label element:", label_element.size())

#     optimizer.zero_grad() 
#     output_element = net(data_element)

#     loss = criterion(output_element,label_element)
#     loss.backward() 
#     optimizer.step()

#     running_loss += loss.data.item()
#     elapsed_time = time.time() - start_time;
#     print("Loss:{}, expected loss:{}".format(running_loss, np.log(3)))
#     print("Time needed:{}s".format(elapsed_time))
#     print("Expected loss for total training data: ", (len(dataset)-split)*np.log(3))
#     print("Expected training time per epoch:{} min".format(elapsed_time* len(train_loader)*TIME_STEPS/BATCH_SIZE/60))
#     print("Estimated total training time:{} hours".format(elapsed_time* len(train_loader)*TIME_STEPS/BATCH_SIZE*NR_EPOCHS/3600))
# print("done")

In [ ]:
print("Start training...")
for epoch in range(NR_EPOCHS): 
    start_time = time.time()
#     print("Epoch:", epoch)
    running_loss = 0.0
    running_test_loss = 0.0
    for i, batch in enumerate(train_loader):
        inputs, labels = batch
        data_in_list = [s.to(device) for s in inputs['flows']]
        labels = labels.to(device)
        
#         print("--------------------------\nOriginal input:")
#         print("Data in: List length", len(data_in_list), "List element:", data_in_list[0].size())
#         print("Labels:", labels.size())

        for j, data_element in enumerate(data_in_list):
            if (labels.size(1) != len(data_in_list)):
                print("WARNING, labels and data_element length do not match in Set",i, "element", j)
#                 print("Label element after")
#                 print(labels[0:len(data_in_list),j].size())
#                 print(labels.size(),len(data_in_list))
            label_element = labels[0:len(data_in_list),j].long()
#             print("--------------------------\nSingle element:")
#             print("Data element size:", data_element.size())
#             print("Label element size:", label_element.size())
#             print("Data element", data_element)
#             print("Label element", label_element)
#             print("Label element after",label_element)

#             print("Data element", data_element[0][0])
    
            optimizer.zero_grad() 
            output_element = net(data_element)
#             print("Output element",output_element)
#             print("Label element after",label_element)
            loss = criterion(output_element,label_element)
            loss.backward() 
            optimizer.step()
            
            running_loss += loss.data.item()/(len(dataset)*(1-VALIDATION_SPLIT))/TIME_STEPS*BATCH_SIZE
#             print("losses:",loss.data.item(),len(dataset),TIMESTEPS)
#             print("Prediction:", torch.max(output_element,1)[1])
#             print("Labels:", label_element.long())
            prediction = torch.max(output_element,1)[1]
            tt_acc_tracker.update_acc(prediction,label_element,"TRAIN")
        
    #TEST SET
    for i,batch in enumerate(test_loader):
        inputs, labels = batch
        data_in_list = [s.to(device) for s in inputs['flows']]
        labels = labels.to(device)
        for j, data_element in enumerate(data_in_list):
            if (labels.size(1) != len(data_in_list)):
                print("WARNING, labels and data_element length do not match in TEST Set",i, "element", j)
#                 print(labels[0:len(data_in_list),j].size())
            label_element = labels[0:len(data_in_list),j].long()

            output_element = net(data_element)
            test_loss = criterion(output_element,label_element)
            running_test_loss += loss.data.item()/(len(dataset)*VALIDATION_SPLIT)/TIME_STEPS

            prediction = torch.max(output_element,1)[1]
            tt_acc_tracker.update_acc(prediction,label_element,"TEST")
        
            
    print("-----------------------------")
    print("Training set:")
    print("Epoch {}: Loss [{},{}], Acc [{},{}] took {}s".format(epoch, running_loss,running_test_loss,tt_acc_tracker.get_acc_tot("TRAIN"),tt_acc_tracker.get_acc_tot("TEST"), np.around(time.time()-start_time,decimals=2)))
    print("Accuracy by class [{},{}] Total elements: [{},{}]".format(tt_acc_tracker.get_acc("TRAIN"),tt_acc_tracker.get_acc("TEST"), tt_acc_tracker.get_labels_distribution("TRAIN"),tt_acc_tracker.get_labels_distribution("TEST")))

    tt_acc_tracker.update_graph()
    tt_acc_tracker.reset_acc_both()
    tt_acc_tracker.update_loss(running_loss)
    tt_acc_tracker.update_lr(LR)
    

    
#     monitor.update_weights(net,['conv5','fc1'])

#test_all_preds(test_net) 
print('...Training finished')
# plt.plot(loss_array)

In [ ]:
NR_EPOCHS = 10
LR = 0.0001

In [ ]:
for i in range(39,40):
    print(i)

In [ ]:
# torch.save(net.state_dict(), '/mnt/DATA/HIWI/IBT/saved_models/PretrainConvFlow/001_all_seq_ts40_test.pt')
# acc_tracker.write_to_csv('/mnt/DATA/HIWI/IBT/saved_models/PretrainConvFlow/001_all_seq_ts40_test/')
# Later:
# loaded_model = PretrainConvFlow.PretrainConvFlow()
# loaded_model.load_state_dict(torch.load('/mnt/DATA/HIWI/IBT/saved_models/PretrainConvFlow/first_draft'))

In [ ]:
# import csv
# path = '/mnt/DATA/HIWI/IBT/saved_models/PretrainConvFlow/001_all_seq_ts40/'
# with open(path + "acc_tot.csv", 'w') as file:
#     wr = csv.writer(file)
#     wr.writerow(acc_tracker.get_acc_tot_graph())
# with open(path + "loss.csv", 'w') as file:
#     wr = csv.writer(file)
#     wr.writerow(acc_tracker.get_loss_graph())
# with open(path + "lr.csv", 'w') as file:
#     wr = csv.writer(file)
#     wr.writerow(acc_tracker.get_lr_graph())
# acc = acc_tracker.get_acc_graph()
# for i in range(len(acc)):
#     with open(path + "acc" + str(i) + ".csv", 'w') as file:
#         wr = csv.writer(file)
#         wr.writerow(acc[i])

In [ ]:
# cnn_flows_pretrain = {
#     'indexing':{
#         #'grouping': 'person_sequence_angle',
#         'selection': 'manual_people_sequence',     #  => 'auto'= by final annotation or
#                                  #  => 'manual_people' = uses 'people' list
#                                  #  => 'manual_people_sequence' uses combination of two lists 'people' and 'sequences'
#         'people_selection': [1],
#         # 'sequences_selection': ['nm-01']
#         'sequences_selection': ['bg-01','bg-02','cl-01','cl-02','nm-01','nm-02','nm-03','nm-04','nm-05','nm-06']
#         },
#     'pose': {
#         'load': False,
#         'preprocess': False,
#         'D': 2 ,
#         # the complete list is:
#         #'body_keypoints_include_list': ['LAnkle' , 'RAnkle' , 'LKnee' , 'RKnee' , 'RHip' , 'LHip' , 'RBigToe' ,
#         #                                'LBigToe' , 'RSmallToe' , 'LSmallToe' , 'RHeel' , 'LHeel']
#         'body_keypoints_include_list': ['LAnkle','RAnkle','LKnee','RKnee','RHip','LHip']
#         },
#     'flow': {
#         'load':True,
#         'preprocess' : True,
#         'crops': True,
#         'method' : 'dense',
#         'load_patches' : True,
#         'patch_size' : 5
#         },
#     'scenes':{
#         'load':False,
#         'preprocess': False,
#         'crops' : False,
#         'gray_scale' : False,
#         'load_tracked' : False,
#         'sequences': ['nm'],
#         'angles': [90]
#     },
#     'heatmaps':{
#         'load':False,
#         'preprocess': False,
#         'body_keypoints_include_list' : ['LAnkle','RAnkle']
#     },
#     'dataset_output' : {
#         'data': ["flows"],
#         'label': "annotations"
#     },
#     'transformers':{
#         # 'Crop':{'include list':['LAnkle','RAnkle'],'output_size':256,'target':'flows'}
#         # 'SpanImagesList': {'remove':True, 'names': ["heatmaps_LAnkle","heatmaps_RAnkle"],'target': ["heatmaps"]},
#         # 'Rescale': {'output_size' : (640,480), 'target': ["flows"]},
#         'AnnotationToLabel': {'target': ["annotations"]},
#         'Transpose' : {'swapping': (2, 0, 1) , 'target': ["flows"]},
#         'Normalize': {'target': ["flows"]},
#         'DimensionResize' : {'start': 5, 'dimension': 40, 'target': ["flows","annotations"],'annotations_offset': 1},
#         'ToTensor': {'target':["flows","annotations"]}
#     }
# }